In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from local_plot import *
from utils import *


In [18]:
nodes = [1, 2, 3]
output_folder = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2vins-3/swarm1"
data_folder="/home/xuhao/data/d2slam/tum_datasets/"
dte = 30
paths_gt, t0 = read_paths(data_folder, nodes, prefix="groundtruth")
paths_vo, t0 = read_paths(data_folder, nodes, prefix="vins-mono", t0=t0)
paths, _ = read_paths(output_folder, nodes, t0=t0)
align_paths(paths_vo, paths_gt)
display(plot_fused_err(nodes, paths, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths, poses_gt=paths_gt, figsize=(10,10))

print("VINS-Mono")
display(plot_fused_err(nodes, paths_vo, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths_vo, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths_vo, poses_gt=paths_gt, figsize=(10,10))


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m,PGO:ATE Pos,ATE Att
1by1,18.1m,0.031,1.080,4.7e-04,1.1e-04,8.2e-05,1.45e-02,nan,nan°
2by1,20.3m,0.029,0.833,1.5e-04,9.9e-04,5.3e-05,2.47e-02,nan,nan°
3by1,16.4m,0.029,1.045,8.2e-04,1.1e-04,1.2e-04,5.99e-03,nan,nan°
Avg.,18.3,0.030,0.99,,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.019,0.012,0.009",0.024,0.70°,"-0.003,-0.001,+0.001",+0.02°
1->3,"0.016,0.017,0.011",0.025,0.85°,"0.004,+0.004,-0.001",-0.07°
Avg:,,0.025,0.77°,,


VINS-Mono


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m,PGO:ATE Pos,ATE Att
1by1,18.2m,0.056,2.659,1.2e-04,6.6e-05,4.8e-05,1.03e-02,nan,nan°
2by1,20.0m,0.052,1.425,1.5e-04,1.1e-04,4.0e-05,7.07e-03,nan,nan°
3by1,16.0m,0.208,6.331,5.5e-04,8.7e-04,5.9e-05,4.67e-03,nan,nan°
Avg.,18.1,0.105,3.47,,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.035,0.059,0.014",0.070,2.70°,"-0.005,+0.044,-0.005",-2.67°
1->3,"0.075,0.165,0.018",0.182,4.13°,"-0.007,-0.139,-0.017",+4.11°
Avg:,,0.126,3.41°,,


In [7]:
nodes = [1, 2, 3, 4]
output_folder = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2vins-4/swarm1"
data_folder="/home/xuhao/data/d2slam/tum_datasets/"
dte = 30
paths_gt, t0 = read_paths(data_folder, nodes, prefix="groundtruth")
paths_vo, t0 = read_paths(data_folder, nodes, prefix="vins-mono", t0=t0)
paths, _ = read_paths(output_folder, nodes, t0=t0)
align_paths(paths_vo, paths_gt)
display(plot_fused_err(nodes, paths, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths, poses_gt=paths_gt, figsize=(10,10))

print("VINS-Mono")
display(plot_fused_err(nodes, paths_vo, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths_vo, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths_vo, poses_gt=paths_gt, figsize=(10,10))


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m,PGO:ATE Pos,ATE Att
1by1,18.0m,0.056,2.138,8.4e-05,6.9e-05,1.4e-04,1.70e-02,nan,nan°
2by1,20.4m,0.066,2.153,9.6e-05,1.6e-04,6.5e-05,7.96e-03,nan,nan°
3by1,16.3m,0.055,2.340,2.7e-04,9.0e-05,1.2e-04,1.08e-02,nan,nan°
4by1,18.6m,0.053,2.187,1.0e-04,1.4e-04,2.1e-04,8.28e-03,nan,nan°
Avg.,18.3,0.057,2.20,,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.016,0.015,0.011",0.024,0.68°,"-0.004,+0.004,-0.002",+0.10°
1->3,"0.011,0.010,0.013",0.019,0.89°,"0.002,+0.003,+0.000",+0.03°
1->4,"0.007,0.008,0.011",0.015,0.65°,"-0.000,+0.001,-0.001",+0.01°
Avg:,,0.020,0.74°,,


VINS-Mono


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m,PGO:ATE Pos,ATE Att
1by1,18.2m,0.056,2.659,1.2e-04,6.6e-05,4.8e-05,1.03e-02,nan,nan°
2by1,20.0m,0.052,1.425,1.5e-04,1.1e-04,4.0e-05,7.07e-03,nan,nan°
3by1,16.0m,0.208,6.331,5.5e-04,8.7e-04,5.9e-05,4.67e-03,nan,nan°
4by1,18.2m,0.117,5.631,7.0e-04,7.2e-04,8.6e-05,1.72e-02,nan,nan°
Avg.,18.1,0.108,4.01,,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.035,0.059,0.014",0.070,2.70°,"-0.005,+0.044,-0.005",-2.67°
1->3,"0.075,0.165,0.018",0.182,4.13°,"-0.007,-0.139,-0.017",+4.11°
1->4,"0.033,0.072,0.036",0.088,1.44°,"-0.011,-0.067,+0.035",-1.37°
Avg:,,0.113,2.76°,,


In [19]:
nodes = [1, 2, 3, 4, 5]
output_folder = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2vins-5/swarm1"
data_folder="/home/xuhao/data/d2slam/tum_datasets/"
dte = 30
paths_gt, t0 = read_paths(data_folder, nodes, prefix="groundtruth")
paths_vo, t0 = read_paths(data_folder, nodes, prefix="vins-mono", t0=t0)
paths, _ = read_paths(output_folder, nodes, t0=t0)
align_paths(paths_vo, paths_gt)
display(plot_fused_err(nodes, paths, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths, poses_gt=paths_gt, figsize=(10,10))

print("VINS-Mono")
display(plot_fused_err(nodes, paths_vo, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths_vo, paths_gt, dte=dte, show=False))
# plot_fused(nodes, paths_vo, poses_gt=paths_gt, figsize=(10,10))


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m,PGO:ATE Pos,ATE Att
1by1,18.0m,0.050,1.265,2.3e-04,9.7e-05,3.5e-04,1.09e-02,nan,nan°
2by1,22.5m,0.067,1.031,2.6e-03,2.9e-04,7.5e-04,8.97e-03,nan,nan°
3by1,18.7m,0.059,1.307,9.6e-04,9.7e-04,3.4e-04,8.99e-03,nan,nan°
4by1,20.6m,0.128,1.684,7.2e-04,1.2e-04,3.6e-04,7.80e-03,nan,nan°
5by1,16.0m,0.092,1.447,5.3e-03,1.9e-03,5.4e-03,2.76e-02,nan,nan°
Avg.,19.2,0.079,1.35,,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.042,0.024,0.019",0.052,0.77°,"0.002,+0.006,+0.003",+0.10°
1->3,"0.023,0.014,0.018",0.033,0.80°,"0.002,-0.001,-0.003",+0.14°
1->4,"0.028,0.010,0.018",0.035,0.71°,"0.003,+0.000,-0.003",-0.01°
1->5,"0.028,0.009,0.018",0.034,0.64°,"0.008,+0.001,+0.001",-0.05°
Avg:,,0.039,0.73°,,


VINS-Mono


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m,PGO:ATE Pos,ATE Att
1by1,18.2m,0.056,2.659,1.2e-04,6.6e-05,4.8e-05,1.03e-02,nan,nan°
2by1,20.0m,0.052,1.425,1.5e-04,1.1e-04,4.0e-05,7.07e-03,nan,nan°
3by1,16.0m,0.208,6.331,5.5e-04,8.7e-04,5.9e-05,4.67e-03,nan,nan°
4by1,18.2m,0.117,5.631,7.0e-04,7.2e-04,8.6e-05,1.72e-02,nan,nan°
5by1,13.8m,0.071,6.646,1.3e-03,4.8e-04,1.8e-03,7.23e-03,nan,nan°
Avg.,17.3,0.101,4.54,,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.035,0.059,0.014",0.070,2.70°,"-0.005,+0.044,-0.005",-2.67°
1->3,"0.075,0.165,0.018",0.182,4.13°,"-0.007,-0.139,-0.017",+4.11°
1->4,"0.033,0.072,0.036",0.088,1.44°,"-0.011,-0.067,+0.035",-1.37°
1->5,"0.083,0.044,0.023",0.097,5.44°,"-0.080,-0.007,-0.022",-5.42°
Avg:,,0.109,3.43°,,


In [20]:
nodes = [1, 2, 3, 4, 5]
output_folder = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2vins-5-async/swarm1"
data_folder="/home/xuhao/data/d2slam/tum_datasets/"
dte = 30
paths_gt, t0 = read_paths(data_folder, nodes, prefix="groundtruth")
paths, _ = read_paths(output_folder, nodes, t0=t0)
display(plot_fused_err(nodes, paths, paths_gt, dte=dte, show=False))
display(plot_relative_pose_err(1, nodes[1:], paths, paths_gt, dte=dte, show=False))

Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m,PGO:ATE Pos,ATE Att
1by1,18.1m,0.106,4.435,3.3e-04,1.7e-04,2.7e-04,1.39e-01,nan,nan°
2by1,21.6m,0.107,4.282,3.0e-04,3.2e-04,1.4e-03,4.17e-02,nan,nan°
3by1,17.6m,0.123,4.958,6.3e-04,4.8e-04,7.5e-04,3.42e-02,nan,nan°
4by1,19.3m,0.108,4.374,2.8e-04,3.2e-04,7.3e-04,3.63e-02,nan,nan°
5by1,16.1m,0.095,4.469,1.1e-03,4.9e-04,1.1e-03,3.09e-02,nan,nan°
Avg.,18.5,0.108,4.50,,,,,,


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
1->2,"0.035,0.026,0.025",0.050,1.17°,"-0.011,+0.002,+0.005",+0.34°
1->3,"0.032,0.023,0.023",0.045,1.10°,"-0.005,-0.005,+0.001",+0.38°
1->4,"0.009,0.011,0.021",0.026,0.70°,"-0.001,-0.001,+0.003",+0.09°
1->5,"0.009,0.008,0.023",0.026,0.59°,"0.001,+0.003,+0.002",+0.08°
Avg:,,0.037,0.89°,,


In [7]:
nodes = [1, 2, 3, 4, 5]
output_folder = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2slam/swarm"
data_folder="/home/xuhao/data/d2slam/tum_datasets/"
dte = 550
paths, paths_pgo, t0 = read_multi_folder(output_folder, nodes)
paths_gt, _ = read_paths(data_folder, nodes, prefix="groundtruth_", t0=t0)
align_paths(paths, paths_gt, align_by_first=True)
if paths_pgo is not None:
    align_paths(paths_pgo, paths_gt, align_by_first=True, align_with_minize=True)
display(plot_fused_err(nodes, paths, paths_gt, poses_pgo=paths_pgo, dte=dte, show=False))
display(relative_pose_err(nodes, paths, paths_gt, dte=dte))

[  6.15027261   6.25027966   6.4504118  ... 138.8546164  139.05462241
 139.30456614]


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m,PGO:ATE Pos,ATE Att
1,145.4m,0.126,1.563,3.4e-04,4.0e-04,1.7e-04,5.99e-03,0.025,0.718°
2,140.9m,0.158,3.113,3.2e-03,1.8e-03,9.2e-05,5.70e-03,0.027,0.710°
3,136.0m,0.137,5.789,1.2e-02,1.8e-03,2.3e-04,1.12e-01,0.024,0.743°
4,67.7m,0.096,2.193,3.7e-04,6.0e-04,4.8e-04,7.43e-03,0.046,0.657°
5,129.1m,0.146,1.947,1.5e-03,7.7e-04,5.6e-04,2.33e-02,0.039,0.749°
Avg.,123.8,0.133,2.92,,,,,0.032,0.72


Relative,EST RMSE: Pos (XYZ),POS,Ang,BIAS: Pos,Ang
2->1,"0.040,0.047,0.027",0.067,1.53°,"0.009,+0.003,-0.002",+0.12°
3->1,"0.063,0.044,0.032",0.083,2.39°,"-0.002,-0.009,-0.002",-0.57°
3->2,"0.079,0.054,0.025",0.099,2.19°,"0.009,-0.011,+0.000",+0.86°
4->1,"0.031,0.039,0.034",0.060,1.42°,"0.002,+0.013,-0.001",-0.29°
4->2,"0.045,0.050,0.033",0.075,1.22°,"0.013,+0.012,-0.003",+0.03°
4->3,"0.063,0.037,0.043",0.085,1.91°,"-0.005,+0.009,+0.002",-2.60°
5->1,"0.044,0.049,0.033",0.074,2.03°,"0.001,+0.006,-0.007",-0.01°
5->2,"0.050,0.051,0.027",0.076,1.87°,"-0.007,+0.005,-0.006",+0.19°
5->3,"0.068,0.054,0.032",0.092,2.57°,"0.005,+0.014,-0.006",+1.29°
5->4,"0.045,0.048,0.039",0.076,1.86°,"-0.005,+0.012,-0.004",+0.09°
